In [ ]:
# !pip install comfit -q # Uncomment and run this cell if you are using Google Colab

If you find any errors in this tutorial, please raise a issue at [GitHub](https://github.com/vidarsko/ComFiT/issues).

# 2.2 - BoseEinsteinCondensate tutorial: Time dependent potentials

In this notebook we are going to illustrate how the potential works in the BoseEinsteinCondensate module. We start by initialising a 2 dimensional BoseEinsteinCondensate with dimensions 101x101. The factor bec.gamma is here set to be non-zero to introduce some dissipation to the model. This is important because the bec with $\gamma = 0$ conserves energy, so if you stir it long enought with a potential it will diverge.


In [ ]:
import comfit as cf
import matplotlib.pyplot as plt
import numpy as np

### Task 1: initialise a bec in two dimensions with resolution 101 in x and y. Make gamma = 0.05 
bec = cf.BoseEinsteinCondensate(2,xRes=101,yRes=101,gamma=0.05,dt=0.1, plot_lib = 'matplotlib')


Now we need to initialize the wavefunction. Before we do that we need to specify the potential. The function
bec.conf_external_potential(V_ext, additive=False), which was used in the previus tutorial can be used when V_ext is a  constant, a matrix or a function. If additive = True the function adds V_ext to the existing potential. 




In [ ]:
### First we set the size of the harmonic
R_tf = 40

### Here we set the size and velocity of the stirrer
stirrer_radius = 20
stirrer_velocity = 0.6
freq = stirrer_velocity/stirrer_radius
size =4
strength = .9

### Defining the function for the time-dependent potential
def V_t(t):
    pos_x = bec.xmid + stirrer_radius * np.cos(freq * t)
    pos_y = bec.ymid + stirrer_radius * np.sin(freq * t)
    stirrer = bec.calc_gaussian_stirring_potential(size, strength, [pos_x, pos_y])
    harmonic = bec.calc_harmonic_potential(R_tf)
    return   harmonic + stirrer


In [ ]:
### Task 2: Set the potential to the above function, initialise the Thomas Fermi 
# ground state and relax the system using the  evolve_relax(...) solver for 20 time steps

bec.conf_external_potential(V_t, additive=False)

bec.conf_initial_condition_Thomas_Fermi()

bec.evolve_relax(50) 

bec.plot_field(np.abs(bec.psi)**2,cmap_symmetric=False,colormap = 'winter')
plt.show()

Note that when the evolve_relax(...) function is running the potential is kept constant and the bec.time variable is not updated

In [ ]:


### Task 4: Evolve the system with the time-dependent potential using the ETD4RK scheme

bec.evolve_dGPE( 30, method='ETD4RK') 



### Task 5: Track the defects and their velocity and plot the result 

nodes = bec.calc_vortex_nodes()

fig,ax=bec.plot_field(np.abs(bec.psi)**2,cmap_symmetric=False,colormap = 'gray')
bec.plot_nodes(nodes,ax=ax)
plt.show()

Now we set the potential to be time independent and run the system again. The non-zero bec.gamma are going to relax the system.


When working with a time dependent sysytem it is nice to make some movies. To do this one needs to use two functions. The first one is cf.plot_save(n,fig) wich saves the plot and label it as n. When all the plots is saved you can cal cf.tool_make_animation(N-1) which takes the figures labeled 0 - (N-1) and makes a plot. It also deletes the figures. The procedure for making a movie is therefore

for i in range(N):

    evolve(...)
    
    make_plot(...)
    
    cf.tool_save_plot(i)
   
cf.tool_make_animation(i) (notice indent)



In [ ]:
#### task 6. make an animation of the stirring potential. Evolve 10 or 20 timesteps between each frame for a total 
#### of 3000 or more timesteps.
#### Display both the absolute value squared of the wavefunction and track the vortices.  Notice that in
###  order to make the plots apear in the same axes you need to use:
###  fig,ax = bec.plot_field(...)
###  bec.plot_nodes(nodes, ax) 

N = 300
for n in range(N):
    bec.evolve_dGPE(10)
    fig,ax=bec.plot_field(abs(bec.psi)**2,colormap='winter',cmap_symmetric=False,
                clims=[0,1])
    nodes = bec.calc_vortex_nodes()
    bec.plot_nodes(nodes,ax=ax)
    bec.plot_save(n,fig)


cf.tool_make_animation_gif(n)

In [ ]:
const_pot = V_t(bec.time)

bec.conf_external_potential(const_pot, additive=False)

In [ ]:
timesteps = 300
bec.evolve_dGPE(timesteps,'ETD4RK')

timesteps = int(200/bec.dt)
bec.plot_field(np.abs(bec.psi)**2,cmap_symmetric=False,colormap = 'winter')
plt.show()

Task 7 (optional): Do the task again, but implement your own time-dependent potential.  